# 🏆 1등 팀 전력 소비량 예측 파이프라인

**실행 환경**: Google Colab (GPU 필수)

**실행 순서**:
1. 라이브러리 설치
2. 데이터 업로드 (Colab에 data 폴더 업로드)
3. 전처리
4. 모델 학습 (각 섹션 선택 실행 가능)
5. 최종 앙상블

**예상 실행 시간**: 약 3-5시간 (GPU 사용 시)

## 📦 Section 0: 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 필수 라이브러리 설치
!pip install -q tabpfn==2.1.2
!pip install -q tabpfn-extensions==0.1.3
!pip install -q autogluon==1.2
!pip install -q scikit-learn pandas numpy tqdm

In [ ]:
# 기본 라이브러리 임포트
import os
import random
import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from typing import List, Tuple, Optional
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from tabpfn import TabPFNRegressor

warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

In [ ]:
# 랜덤 시드 고정
SEED = 42

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seeds()
print(f"✅ 랜덤 시드 설정: {SEED}")

In [ ]:
# 데이터 경로 설정 (Colab에서는 data 폴더를 업로드하세요)
DATA_DIR = './data'

# 데이터 폴더 구조 확인
if os.path.exists(DATA_DIR):
    print("✅ 데이터 폴더 발견")
    print("\n폴더 내용:")
    for file in os.listdir(DATA_DIR):
        print(f"  - {file}")
else:
    print("⚠️ 데이터 폴더가 없습니다. data 폴더를 업로드하세요.")
    print("\n필요한 파일:")
    print("  - train.csv")
    print("  - test.csv")
    print("  - building_info.csv")
    print("  - sample_submission.csv")

## 📊 Section 1: 전처리 함수 정의

In [ ]:
# 상수 정의
WEEK_H = 168  # 1주일 = 168시간
EPS = 1e-3

# 컬럼명 매핑
TRAIN_COL_RENAMES = {
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
}
TEST_COL_RENAMES = TRAIN_COL_RENAMES.copy()

BUILDING_INFO_RENAMES = {
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
}

TYPE_TRANSLATION = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'School',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '호텔': 'Hotel',
    'IDC(전화국)': 'IDC'
}

KR_HOLIDAYS_2024 = {"2024-06-06", "2024-08-15"}

DROP_COLS = ['sunshine', 'solar_radiation', 'solar_power_capacity', 'ess_capacity', 'pcs_capacity', 
             'hour', 'day_of_week', 'day_of_year']
CAT_COLS = ['building_type', 'building_number']

print("✅ 상수 및 매핑 정의 완료")

In [ ]:
# 데이터 로드 함수
def load_raw(data_dir: str = "./data"):
    train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
    info = pd.read_csv(os.path.join(data_dir, 'building_info.csv'))
    return train, test, info

def rename_columns(df: pd.DataFrame, mapping: dict):
    df = df.rename(columns=mapping)
    if 'num_date_time' in df.columns:
        df = df.drop('num_date_time', axis=1)
    return df

def preprocess_building_info(info: pd.DataFrame) -> pd.DataFrame:
    info = info.rename(columns=BUILDING_INFO_RENAMES)
    info['building_type'] = info['building_type'].replace(TYPE_TRANSLATION)
    return info

def merge_datasets(train: pd.DataFrame, test: pd.DataFrame, info: pd.DataFrame):
    train = train.merge(info, on='building_number', how='left')
    test = test.merge(info, on='building_number', how='left')
    return train, test

print("✅ 데이터 로드 함수 정의 완료")

In [ ]:
# Feature Engineering 함수들
def create_datetime(df: pd.DataFrame) -> pd.DataFrame:
    df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d %H')
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['day_of_year'] = df['date_time'].dt.dayofyear
    return df

def add_summer_cycle_features(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    start_date = datetime.strptime("2024-05-20 00:00:00", "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime("2024-09-08 00:00:00", "%Y-%m-%d %H:%M:%S")
    period_seconds = (end_date - start_date).total_seconds()
    
    def summer_cos(date):
        return np.cos(2 * np.pi * (date - start_date).total_seconds() / period_seconds)
    
    def summer_sin(date):
        return np.sin(2 * np.pi * (date - start_date).total_seconds() / period_seconds)
        
    df_copy['summer_cos'] = df_copy['date_time'].apply(summer_cos)
    df_copy['summer_sin'] = df_copy['date_time'].apply(summer_sin)
    return df_copy

def add_squared_features(df: pd.DataFrame, target_cols: List[str] = ['temperature', 'humidity']) -> pd.DataFrame:
    df_copy = df.copy()
    for col in target_cols:
        df_copy[f'{col}_squared'] = df_copy[col] ** 2
    return df_copy

def create_cyclic_features(df):
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_doy'] = np.sin(2 * np.pi * (df['day_of_year'] - 1) / 365)
    df['cos_doy'] = np.cos(2 * np.pi * (df['day_of_year'] - 1) / 365)
    return df

def cooling_degree_hour(temperature, window=12, base_temp=26):
    cdhs = []
    temps = temperature.values
    for i in range(len(temps)):
        if i < window:
            cdh = np.sum(np.maximum(temps[:i+1] - base_temp, 0))
        else:
            cdh = np.sum(np.maximum(temps[i-window+1:i+1] - base_temp, 0))
        cdhs.append(cdh)
    return cdhs

def add_cdh_feature(df: pd.DataFrame, window: int = 12, base_temp: float = 26.0) -> pd.DataFrame:
    cdhs_all = []
    for b in df['building_number'].unique():
        temps = df.loc[df['building_number'] == b, 'temperature']
        cdhs_all.extend(cooling_degree_hour(temps, window=window, base_temp=base_temp))
    df['CDH'] = cdhs_all
    return df

def add_cdd_feature(df: pd.DataFrame, base_temp: float = 18.0, window: int = 24) -> pd.DataFrame:
    df['excess'] = (df['temperature'] - base_temp).clip(lower=0)
    df['CDD'] = df.groupby('building_number')['excess'].transform(
        lambda s: s.rolling(window, min_periods=1).sum()
    )
    df.drop(columns=['excess'], inplace=True)
    return df

def add_thi_feature(df: pd.DataFrame) -> pd.DataFrame:
    df['THI'] = (9/5 * df['temperature'] 
                 - 0.55 * (1 - df['humidity']/100) 
                 * (9/5 * df['temperature'] - 26) 
                 + 32)
    return df

def add_wct_feature(df: pd.DataFrame) -> pd.DataFrame:
    v16 = df['windspeed'] ** 0.16
    df['WCT'] = (13.12 
                 + 0.6215 * df['temperature'] 
                 - 11.37 * v16 
                 + 0.3965 * v16 * df['temperature'])
    return df

def add_temp_features(data):
    avg_temp = pd.pivot_table(
        data[data['hour'] % 3 == 0],
        values='temperature',
        index=['building_number', 'day', 'month'],
        aggfunc='mean'
    ).reset_index().rename(columns={'temperature': 'avg_temp'})
    data = pd.merge(data, avg_temp, on=['building_number', 'day', 'month'], how='left')

    max_temp = pd.pivot_table(
        data,
        values='temperature',
        index=['building_number', 'day', 'month'],
        aggfunc='max'
    ).reset_index().rename(columns={'temperature': 'max_temp'})
    data = pd.merge(data, max_temp, on=['building_number', 'day', 'month'], how='left')

    min_temp = pd.pivot_table(
        data,
        values='temperature',
        index=['building_number', 'day', 'month'],
        aggfunc='min'
    ).reset_index().rename(columns={'temperature': 'min_temp'})
    data = pd.merge(data, min_temp, on=['building_number', 'day', 'month'], how='left')

    data['temp_diff'] = data['max_temp'] - data['min_temp']
    return data

def add_humid_features(data):
    avg_humid = pd.pivot_table(
        data[data['hour'] % 3 == 0],
        values='humidity',
        index=['building_number', 'day', 'month'],
        aggfunc='mean'
    ).reset_index().rename(columns={'humidity': 'avg_humid'})
    data = pd.merge(data, avg_humid, on=['building_number', 'day', 'month'], how='left')

    max_humid = pd.pivot_table(
        data,
        values='humidity',
        index=['building_number', 'day', 'month'],
        aggfunc='max'
    ).reset_index().rename(columns={'humidity': 'max_humid'})
    data = pd.merge(data, max_humid, on=['building_number', 'day', 'month'], how='left')

    min_humid = pd.pivot_table(
        data,
        values='humidity',
        index=['building_number', 'day', 'month'],
        aggfunc='min'
    ).reset_index().rename(columns={'humidity': 'min_humid'})
    data = pd.merge(data, min_humid, on=['building_number', 'day', 'month'], how='left')

    data['humid_diff'] = data['max_humid'] - data['min_humid']
    return data

def _prep(df, time_col, group_col):
    return df.sort_values([group_col, time_col])

def add_weekly_slope(df: pd.DataFrame, time_col: str = 'date_time',
                     group_col: str = 'building_number',
                     power_col: str = 'power_consumption',
                     lookback: int = 6) -> pd.DataFrame:
    df = _prep(df, time_col, group_col)

    def _beta(x: pd.Series) -> float:
        if x.isna().any(): 
            return np.nan
        idx = np.arange(len(x))
        num = idx.dot(x) * len(x) - idx.sum() * x.sum()
        den = len(x) * (idx**2).sum() - idx.sum()**2
        return num / den if den else 0.0

    pw_seq = df.groupby(group_col)[power_col].shift(WEEK_H)
    col = f'power_week_slope{lookback}h'
    df[col] = pw_seq.groupby(df[group_col]).transform(
        lambda s: s.rolling(lookback).apply(_beta, raw=False)
    ).fillna(0)
    return df

print("✅ Feature Engineering 함수 정의 완료")

In [ ]:
# 휴일 설정 함수 (건물별 세밀한 설정)
def _ensure_dt(df):
    if not np.issubdtype(df["date_time"].dtype, np.datetime64):
        df["date_time"] = pd.to_datetime(df["date_time"])
    return df

def _nth_weekday_in_month(series_dt, weekday_target):
    first_of_month = series_dt.values.astype("datetime64[M]").astype("datetime64[ns]")
    first_weekday = pd.to_datetime(first_of_month).weekday
    weekday = series_dt.dt.weekday.values
    day = series_dt.dt.day.values
    first_occ_day = 1 + ((weekday_target - first_weekday) % 7)
    nth = ((day - first_occ_day) // 7) + 1
    nth = np.where(day >= first_occ_day, nth, 0)
    return nth

def add_holiday(df: pd.DataFrame, kr_holidays: set[str] = None) -> pd.DataFrame:
    df = df.copy()
    _ensure_dt(df)
    if kr_holidays is None:
        kr_holidays = KR_HOLIDAYS_2024

    df["weekday"] = df["date_time"].dt.weekday
    df["date"] = df["date_time"].dt.date
    df["is_weekend"] = (df["weekday"] >= 5).astype(int)
    df["holiday"] = 0

    is_kr = df["date"].astype(str).isin(kr_holidays).values
    bt = df["building_type"]

    # Apartment: 항상 영업
    mm = bt == "Apartment"
    df.loc[mm, "holiday"] = 0

    # Hospital: 주말 or 공휴일 휴식
    mm = bt == "Hospital"
    if mm.any():
        df.loc[mm, "holiday"] = (df.loc[mm, "is_weekend"].values | is_kr[mm]).astype(int)

    # Public: 기본 주말 or 공휴일, 단 33/92는 항상 영업
    mm = bt == "Public"
    if mm.any():
        df.loc[mm, "holiday"] = (df.loc[mm, "is_weekend"].values | is_kr[mm]).astype(int)
        mm_always_open = df["building_number"].isin([33, 92])
        df.loc[mm_always_open, "holiday"] = 0

    # Hotel: 항상 영업
    mm = bt == "Hotel"
    df.loc[mm, "holiday"] = 0

    # School: 주말 or 공휴일 휴식
    mm = bt == "School"
    if mm.any():
        df.loc[mm, "holiday"] = (df.loc[mm, "is_weekend"].values | is_kr[mm]).astype(int)

    # IDC: 개별 규칙
    mm_idc = bt == "IDC"
    if mm_idc.any():
        ids = [36, 43, 52]
        mmx = df["building_number"].isin(ids)
        df.loc[mmx, "holiday"] = (df.loc[mmx, "is_weekend"].values | is_kr[mmx]).astype(int)
        mmx = df["building_number"].eq(64)
        df.loc[mmx, "holiday"] = df.loc[mmx, "is_weekend"].astype(int)
        mmx = df["building_number"].eq(67)
        if mmx.any():
            df.loc[mmx, "holiday"] = df.loc[mmx, "is_weekend"].astype(int)
            df.loc[mmx & (df["date"].astype(str) == "2024-08-15"), "holiday"] = 1

    # Commercial: 개별 규칙
    mm = bt == "Commercial"
    if mm.any():
        mmx = df["building_number"].eq(2)
        df.loc[mmx, "holiday"] = df.loc[mmx, "is_weekend"].astype(int)
        ids = [6, 16, 20, 51, 86]
        mmx = df["building_number"].isin(ids)
        df.loc[mmx, "holiday"] = (df.loc[mmx, "is_weekend"].values | is_kr[mmx]).astype(int)

    # Other Buildings
    mmx = df["building_number"].eq(26)
    df.loc[mmx, "holiday"] = df.loc[mmx, "weekday"].isin([0, 1]).astype(int)
    mmx = df["building_number"].eq(82)
    df.loc[mmx, "holiday"] = df.loc[mmx, "weekday"].eq(0).astype(int)
    mmx = df["building_number"].isin([47, 69])
    df.loc[mmx, "holiday"] = (df.loc[mmx, "is_weekend"].values | is_kr[mmx]).astype(int)
    mmx = df["building_number"].eq(97)
    df.loc[mmx, "holiday"] = df.loc[mmx, "weekday"].eq(5).astype(int)

    # Department Store: 개별 휴일 규칙
    mm = bt == "Department Store"
    if mm.any():
        df.loc[mm, "holiday"] = 0
        nth_sun = _nth_weekday_in_month(df["date_time"], 6)
        nth_mon = _nth_weekday_in_month(df["date_time"], 0)

        def mark_nth_weekday(building, weekday, nth_set):
            if weekday == 6:
                nth = nth_sun
            elif weekday == 0:
                nth = nth_mon
            else:
                nth = _nth_weekday_in_month(df["date_time"], weekday)
            sel = df["building_number"].eq(building) & df["weekday"].eq(weekday) & pd.Series(nth).isin(list(nth_set)).values
            df.loc[sel, "holiday"] = 1

        df.loc[df["building_number"].eq(18) & df["weekday"].eq(6), "holiday"] = 1

        special = {
            19: ["2024-06-10", "2024-07-08", "2024-08-19"],
            45: ["2024-06-10", "2024-07-08", "2024-08-19"],
            54: ["2024-06-17", "2024-07-01", "2024-08-19"],
            74: ["2024-06-17", "2024-07-01"],
            79: ["2024-06-17", "2024-07-01", "2024-08-19"],
            95: ["2024-07-08", "2024-08-05"],
            29: ["2024-06-10", "2024-07-10", "2024-08-10"],
        }
        for b, dates in special.items():
            sel = df["building_number"].eq(b) & df["date"].astype(str).isin(dates)
            df.loc[sel, "holiday"] = 1

        mark_nth_weekday(27, 6, {2, 4})
        mark_nth_weekday(29, 6, {4})
        mark_nth_weekday(32, 0, {2, 4})
        for b in [40, 59, 63]:
            mark_nth_weekday(b, 6, {2, 4})

    # Research Institute
    mm = bt == "Research Institute"
    if mm.any():
        df.loc[mm, "holiday"] = (df.loc[mm, "is_weekend"].values | is_kr[mm]).astype(int)
        
        nth_fri = _nth_weekday_in_month(df["date_time"], 4)
        sel_23 = df["building_number"].eq(23) & mm
        df.loc[sel_23 & df["weekday"].eq(4) & pd.Series(nth_fri).eq(3).values, "holiday"] = 1
        extra_23 = {"2024-06-07", "2024-08-16"}
        df.loc[sel_23 & df["date"].astype(str).isin(extra_23), "holiday"] = 1

        sel_49 = df["building_number"].eq(49) & mm
        df.loc[sel_49 & df["date"].astype(str).eq("2024-08-22"), "holiday"] = 1

        sel_53 = df["building_number"].eq(53) & mm
        extra_53 = {"2024-06-15", "2024-06-16"}
        df.loc[sel_53 & df["date"].astype(str).isin(extra_53), "holiday"] = 1

        sel_94 = df["building_number"].eq(94) & mm
        extra_94 = {"2024-06-07", "2024-08-16"}
        df.loc[sel_94 & df["date"].astype(str).isin(extra_94), "holiday"] = 1

    df.loc[(df["building_number"].eq(67)) & (df["date"].astype(str) == "2024-08-15"), "holiday"] = 1

    return df

print("✅ 휴일 설정 함수 정의 완료")

In [ ]:
# 이상치 제거 및 타겟 통계 함수
def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    _ensure_dt(df)

    rules_lt = [
        (25, 0, "eq"), (70, 200, "lt"),
        (44, 800, "lt"), (90, 800, "lt"), (42, 2000, "lt"), (17, 1000, "lt"),
        (68, 600, "lt"), (72, 600, "lt"), (80, 600, "lt"), (92, 200, "lt"),
        (98, 500, "lt"),
        (97, 500, "lt"), (78, 400, "lt"), (26, 300, "lt"), (7, 2000, "lt"),
        (76, 2000, "lt"), (41, 2200, "lt"), (20, 1600, "lt"),
        (5, 2000, "lt"), (8, 250, "lt"), (12, 3500, "lt"),
        (67, 7333, "lt"), (81, 800, "lt"), (52, 2000, "lt"), (43, 6000, "lt"), (30, 8000, "lt"),
    ]

    mask_ok = pd.Series(True, index=df.index)
    pc = df["power_consumption"]
    bnum = df["building_number"]

    for bn, th, op in rules_lt:
        if op == "lt":
            mask_ok &= ~((bnum.eq(bn)) & (pc < th))
        elif op == "eq":
            mask_ok &= ~((bnum.eq(bn)) & (pc == th))

    mask_ok &= ~((bnum.eq(10)) & (df["date_time"].between(pd.Timestamp("2024-07-05"), pd.Timestamp("2024-08-22"))))
    mask_ok &= ~((bnum.eq(57)) & (df["date_time"] < pd.Timestamp("2024-06-07")))
    mask_ok &= ~((bnum.eq(94)) & (df["date_time"].between(pd.Timestamp("2024-07-27 09:00"), pd.Timestamp("2024-08-04 23:00"))))
    mask_ok &= ~((bnum.eq(53)) & (df["date_time"].dt.normalize().isin([pd.Timestamp("2024-06-15"), pd.Timestamp("2024-06-16")])))
    mask_ok &= ~((bnum.eq(53)) & (df["date_time"] >= pd.Timestamp("2024-08-17")) & (pc <= 1000))

    return df.loc[mask_ok].reset_index(drop=True)

def mean_std_power(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    is_train = df['power_consumption'].notna()

    dt = pd.to_datetime(df['date_time'])
    if 'date' not in df.columns:
        df['date'] = dt.dt.date
    if 'hour' not in df.columns:
        df['hour'] = dt.dt.hour
    if 'day_of_week' not in df.columns:
        df['day_of_week'] = dt.dt.weekday
    if 'month' not in df.columns:
        df['month'] = dt.dt.month

    df['holiday'] = df['holiday'].fillna(0).astype(int)

    base_ratio = np.array([1.0] * 7)
    ratio_all = base_ratio - 0
    df.loc[is_train, 'power_consumption'] = df.loc[is_train].apply(
        lambda r: r['power_consumption'] * ratio_all[int(r['day_of_week'])],
        axis=1
    )

    train_df = df[is_train].copy()

    PUBLIC_HOLS = {"2024-06-06", "2024-08-15"}
    train_df_dow = train_df[~train_df['date'].astype(str).isin(PUBLIC_HOLS)].copy()

    dow_hour_mean = train_df_dow.groupby(['building_number', 'hour', 'day_of_week'])['power_consumption'].mean().reset_index(name='dow_hour_mean')
    dow_hour_std = train_df_dow.groupby(['building_number', 'hour', 'day_of_week'])['power_consumption'].std().reset_index(name='dow_hour_std')
    df = df.merge(dow_hour_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
    df = df.merge(dow_hour_std, on=['building_number', 'hour', 'day_of_week'], how='left')

    hol_mean = train_df.groupby(['building_number', 'hour', 'holiday'])['power_consumption'].mean().reset_index(name='holiday_mean')
    hol_std = train_df.groupby(['building_number', 'hour', 'holiday'])['power_consumption'].std().reset_index(name='holiday_std')
    df = df.merge(hol_mean, on=['building_number', 'hour', 'holiday'], how='left')
    df = df.merge(hol_std, on=['building_number', 'hour', 'holiday'], how='left')

    hr_mean = train_df.groupby(['building_number', 'hour'])['power_consumption'].mean().reset_index(name='hour_mean')
    hr_std = train_df.groupby(['building_number', 'hour'])['power_consumption'].std().reset_index(name='hour_std')
    df = df.merge(hr_mean, on=['building_number', 'hour'], how='left')
    df = df.merge(hr_std, on=['building_number', 'hour'], how='left')

    mh_mean = train_df.groupby(['building_number', 'month', 'hour'])['power_consumption'].mean().reset_index(name='month_hour_mean')
    mh_std = train_df.groupby(['building_number', 'month', 'hour'])['power_consumption'].std().reset_index(name='month_hour_std')
    df = df.merge(mh_mean, on=['building_number', 'month', 'hour'], how='left')
    df = df.merge(mh_std, on=['building_number', 'month', 'hour'], how='left')

    return df

print("✅ 이상치 제거 및 타겟 통계 함수 정의 완료")

## 🔄 Section 2: 데이터 전처리 실행

In [ ]:
# 데이터 로드 및 기본 전처리
print("📥 데이터 로드 중...")
train, test, info = load_raw(DATA_DIR)
train = rename_columns(train, TRAIN_COL_RENAMES)
test = rename_columns(test, TEST_COL_RENAMES)
info = preprocess_building_info(info)
train, test = merge_datasets(train, test, info)

print(f"✅ Train shape: {train.shape}")
print(f"✅ Test shape: {test.shape}")
print(f"✅ Building info shape: {info.shape}")

In [ ]:
# Feature Engineering 실행
print("🔧 Feature Engineering 시작...")

train = create_datetime(train)
test = create_datetime(test)

combined_df = pd.concat([train, test], ignore_index=True)
print(f"  ✓ Combined shape: {combined_df.shape}")

combined_df = add_holiday(combined_df)
print("  ✓ Holiday 설정 완료")

combined_df = remove_outliers(combined_df)
print(f"  ✓ 이상치 제거 완료 (shape: {combined_df.shape})")

combined_df = add_squared_features(combined_df)
print("  ✓ Squared features 추가")

combined_df = add_summer_cycle_features(combined_df)
print("  ✓ Summer cycle features 추가")

combined_df = create_cyclic_features(combined_df)
print("  ✓ Cyclic features 추가")

combined_df = add_cdh_feature(combined_df)
print("  ✓ CDH 추가")

combined_df = add_cdd_feature(combined_df)
print("  ✓ CDD 추가")

combined_df = add_thi_feature(combined_df)
print("  ✓ THI 추가")

combined_df = add_wct_feature(combined_df)
print("  ✓ WCT 추가")

combined_df = add_temp_features(combined_df)
print("  ✓ Temperature features 추가")

combined_df = add_humid_features(combined_df)
print("  ✓ Humidity features 추가")

combined_df = mean_std_power(combined_df)
print("  ✓ Target 통계 features 추가")

combined_df = add_weekly_slope(combined_df)
print("  ✓ Weekly slope 추가")

print(f"\n✅ 최종 Feature Engineering 완료! Shape: {combined_df.shape}")
print(f"   총 컬럼 수: {len(combined_df.columns)}")

In [ ]:
# Train/Test 분리 및 전처리
split_date = pd.to_datetime('2024-08-25 00:00:00')

x_full_train = combined_df[combined_df['date_time'] < split_date].copy()
test = combined_df[combined_df['date_time'] >= split_date].copy()

for c in CAT_COLS:
    x_full_train[c] = x_full_train[c].astype('category')
    test[c] = test[c].astype('category')

x_full_train = x_full_train.ffill()

x_full_train.drop(columns=DROP_COLS, inplace=True)
test.drop(columns=DROP_COLS, inplace=True)

print(f"✅ Train shape: {x_full_train.shape}")
print(f"✅ Test shape: {test.shape}")

## 🤖 Section 3: TabPFN Stacking 모델 (건물별)

In [ ]:
# Stacking 함수 정의
def get_stacking_ml_datasets(model, X_train_n, y_train_n, X_test_n, n_folds, seed=42):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n, y_train_n)):
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    return train_fold_pred, test_pred_mean

print("✅ Stacking 함수 정의 완료")

In [ ]:
# 데이터 준비
KI_train = x_full_train.copy()
KI_test = test.copy()

KI_train['건물번호'] = KI_train['building_number']
KI_test['건물번호'] = KI_test['building_number']
KI_train['일시'] = KI_train['date_time']
KI_test['일시'] = KI_test['date_time']
KI_train['전력소비량(kWh)'] = KI_train['power_consumption']

drop_columns = ['building_type', 'total_area', 'cooling_area', 'date']
KI_train = KI_train.drop(drop_columns, axis=1)
KI_test = KI_test.drop(drop_columns, axis=1)

train_df = KI_train.drop(['building_number', 'date_time', 'power_consumption'], axis=1)
test_df = KI_test.drop(['building_number', 'date_time', 'power_consumption'], axis=1)

print(f"✅ Stacking용 데이터 준비 완료")
print(f"   Train features: {len(train_df.columns)}")

In [ ]:
# TabPFN Stacking 모델 학습 (건물별)
# 주의: 이 부분은 실행 시간이 매우 깁니다 (약 1-2시간)

print("🚀 TabPFN Stacking 학습 시작...")
print("⚠️ 예상 소요 시간: 약 45-60분 (GPU 사용 시)")

CV_FOLDS = 10
scaler = StandardScaler()
tabpfn = TabPFNRegressor(random_state=SEED, n_jobs=-1)
best_ml = [tabpfn]

preds_total = []

for b_num in tqdm(train_df['건물번호'].unique(), desc="Building"):
    train_b = train_df[train_df["건물번호"] == b_num]
    test_b = test_df[test_df["건물번호"] == b_num]

    X_train = train_b.drop(['건물번호', '일시', '전력소비량(kWh)'], axis=1)
    y_train = train_b['전력소비량(kWh)'].values
    X_test = test_b.drop(['건물번호', '일시'], axis=1)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    meta_X_train = []
    meta_X_test = []
    
    for estimator in best_ml:
        temp_X_train, temp_X_test = get_stacking_ml_datasets(
            estimator, X_train, y_train, X_test, CV_FOLDS
        )
        meta_X_train.append(temp_X_train)
        meta_X_test.append(temp_X_test)
        
    meta_X_train = np.hstack(meta_X_train)
    meta_X_test = np.hstack(meta_X_test)

    meta_clf = LinearRegression()
    meta_clf.fit(meta_X_train, y_train)
    preds_partial = meta_clf.predict(meta_X_test)
    
    preds_total.append(preds_partial)

prediction_stacking = np.hstack(preds_total)

print(f"\n✅ TabPFN Stacking 완료! Prediction shape: {prediction_stacking.shape}")

In [ ]:
# Stacking 결과 저장
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))
submission['answer'] = np.round(prediction_stacking, 2)
submission.to_csv('stacking_tabpfn.csv', index=False)

print("✅ stacking_tabpfn.csv 저장 완료")
print("\n예측값 샘플:")
print(submission.head())

## 📝 Section 4: 제출 파일 생성

**참고**: 1등 팀은 여러 모델을 앙상블했지만, 여기서는 TabPFN Stacking 결과만 제출합니다.

전체 앙상블을 재현하려면:
1. AutoGluon 모델 추가
2. 건물 타입별 TabPFN 모델 추가
3. 여러 시드로 실행 후 앙상블

이 작업들은 `02_Advanced_Ensemble.ipynb`에서 진행할 수 있습니다.

In [ ]:
# 최종 제출 파일 확인
final_submission = pd.read_csv('stacking_tabpfn.csv')

print("📊 최종 제출 파일 통계:")
print(f"  - 총 예측 개수: {len(final_submission)}")
print(f"  - 예측값 범위: [{final_submission['answer'].min():.2f}, {final_submission['answer'].max():.2f}]")
print(f"  - 예측값 평균: {final_submission['answer'].mean():.2f}")
print(f"  - 예측값 중앙값: {final_submission['answer'].median():.2f}")
print(f"\n결측치 확인: {final_submission.isnull().sum().sum()}")

if final_submission.isnull().sum().sum() == 0:
    print("\n✅ 제출 파일이 정상적으로 생성되었습니다!")
else:
    print("\n⚠️ 결측치가 있습니다. 확인이 필요합니다.")

## 🎯 다음 단계

1. **성능 확인**: `stacking_tabpfn.csv`를 리더보드에 제출
2. **추가 모델**: AutoGluon, 건물 타입별 모델 등 추가 실험
3. **앙상블 최적화**: 여러 모델 조합 및 가중치 최적화
4. **Feature Engineering**: 새로운 피처 추가 실험

**참고**: 
- 1등 팀은 5개 시드의 건물별 모델 + AutoGluon + 타입별 모델을 앙상블
- 전체 재현을 위해서는 약 3-5시간의 학습 시간 필요
- Colab Pro 또는 GPU 환경 권장